In [1]:
"""
Created on Sat Jul 23 22:15:12 2016

@author: A.F., Jul 29 2016
"""
from bokeh.charts import BoxPlot
from bokeh.layouts import row, widgetbox
from bokeh.layouts import column
from bokeh.models import CustomJS, ColumnDataSource, Slider, Range1d, LabelSet, Label
from bokeh.plotting import Figure, output_file, show, output_notebook


import numpy as np


text comment Alex declaring constants

In [2]:
output_notebook()

pi = np.pi
pi = 3.1415

f1 = 5
f2 = 10
f3 = 40
f4 = 45

pif1 = pi*f1
pif2 = pi*f2
pif3 = pi*f3
pif4 = pi*f4

a1 = pif4 * pif4 / (pif4  - pif3)
a2 = pif3 / (pif4/pif3 - 1)
a3 = pif2 / (1 - pif1/pif2)
a4 = pif1 / (pif2/pif1 - 1)

Loading BokehJS ...

In [3]:
#output_file("callback.html", title="ORMSBY wavelet")



x = [-0.2 + x*0.001 for x in range(0, 400)]
y = x
x = np.array(x)
#x1 = x
x1 = x*1000

#for i in range(len(x)):
#    y[i] = a1 * np.sinc(pif4 * x[i]) ** 2 - a2 * np.sinc(pif3 * x[i]) ** 2 - a3 * np.sinc(pif2 * x[i]) ** 2 + a4 * np.sinc(pif1 * x[i]) ** 2

scal = 1. 
for i in range(len(x)):
    t = x[i]
             
    b1 = np.sinc(pif4 * t / pi)
    b2 = np.sinc(pif3 * t / pi)
    b3 = np.sinc(pif2 * t / pi)
    b4 = np.sinc(pif1 * t / pi)
    y[i] = a1 * b1 * b1 - a2 * b2 * b2 - a3 * b3 * b3 + a4 * b4 * b4
    
    if np.abs(y[i]) > scal:
        scal = np.abs(y[i])

#scalar = 1/np.amax(y)
for i in range(len(x)):
    y[i] = scal * y[i]


#for i in range(len(x)):
#    y[i] = (1 - 2*pi*pi*f*f*x[i]*x[i]) * math.exp(-pi*pi*f*f*x[i]*x[i])

source = ColumnDataSource(data=dict(x=x1, y=y))


plot = Figure(title="    ORMSBY wavelet defined by four frequencies f1 - f2 - f3 - f4 (Hz):", plot_width=800, plot_height=400)

plot.line('x', 'y', source=source, line_width=3, line_alpha=0.6)


callback = CustomJS(args=dict(source=source), code="""
        function sinc(a) {
            if (a === 0) {
                return 1;
                } 
            else {
                return Math.sin(a) / a;
                }          
            } 

        var data = source.get('data');
        
        var f1 = sl_f1.get('value')             
        var f2 = sl_f2.get('value')
        if (f1 > f2)  {f2=f1}
        
        var f3 = sl_f3.get('value')     
        if (f3 < f2) { f2 = f3 }  
        
        var f4 = sl_f4.get('value')
        if (f3 > f4) {f4 = f3}

                
        var pi = 3.1415
            
        var pif1 = pi*f1
        var pif2 = pi*f2
        var pif3 = pi*f3
        var pif4 = pi*f4

        var a1 = pif4 / (1 - pif3/pif4)
        var a2 = pif3 / (pif4/pif3 - 1)
        var a3 = pif2 / (1 - pif1/pif2)
        var a4 = pif1 / (pif2/pif1 - 1)
        
        x = data['x']
        y = data['y']

        var scal = 1.        
        
        for (i = 0; i < x.length; i++) {
            t = x[i]/1000.
             
            var b1 = sinc(pif4 * t)
            var b2 = sinc(pif3 * t)
            var b3 = sinc(pif2 * t)
            var b4 = sinc(pif1 * t)
            y[i] = a1 * b1 * b1 - a2 * b2 * b2 - a3 * b3 * b3 + a4 * b4 * b4
            if (Math.abs(y[i]) > scal) {
                scal = Math.abs(y[i])
                }
        }
        
        for (i = 0; i < x.length; i++) {
            y[i] = y[i] / scal
        }
        
        
        source.trigger('change');
        
    """)

slider_f1 = Slider(start=2, end=200, value=5, step=.1, 
                title="f1", callback=callback)
callback.args["sl_f1"] = slider_f1              

slider_f2 = Slider(start=10, end=200, value=10, step=.1, 
                title="f2", callback=callback)
callback.args["sl_f2"] = slider_f2                

slider_f3 = Slider(start=20, end=200, value=40, step=.1, 
                title="f3", callback=callback)
callback.args["sl_f3"] = slider_f3

slider_f4 = Slider(start=20, end=200, value=45, step=.1, 
                title="f4", callback=callback)
callback.args["sl_f4"] = slider_f4

#layout = column(slider_f2, slider_f3, plot)

In [4]:


layout = row(
    plot, 
    widgetbox(slider_f1, slider_f2, slider_f3, slider_f4)
)

show(layout)


# ORMSBY WAVELET